# Neuroevolución y Word Embeddings
- ### La neuroevolución es una técnica de inteligencia artificial que combina redes neuronales con algoritmos evolutivos.
- ### Optimiza automáticamente la estructura y parámetros de una red.

## Inspiración biológica:

- ### Se basa en la evolución natural (selección, mutación y recombinación).

- ### Las redes neuronales "evolucionan" para adaptarse a una tarea.

## Tipos principales:

- ### Evolución de pesos: Optimiza los parámetros: pesos y sesgos ($bias$) de una red fija.

- ### Evolución de topologías: Modifica la arquitectura de la red (Por ejemplo, el método NEAT).

## Ventajas:

- ### No requiere del cálculo de gradientes (útil para problemas no diferenciables).

- ### Explora múltiples soluciones en paralelo.

- ### Puede descubrir arquitecturas novedosas.

## Representación del texto: Word Embeddings

- ### **Word embeddings** son representaciones numéricas densas y continuas de palabras en un espacio vectorial.
- ### Estas representaciones capturan relaciones semánticas y sintácticas entre palabras.
- ### Palabras con significados similares están más cercanas en el espacio vectorial.
- ### Densidad: Cada palabra se representa como un vector en un espacio de dimensiones reducidas (por ejemplo, 100 o 300 dimensiones).
- ### Diferente a las representaciones como las matrices dispersas en el modelo de "bolsa de palabras".
- ### Similitud semántica: Las palabras con significados similares tendrán vectores cercanos en el espacio vectorial.
- ### Por ejemplo, en un buen modelo de embeddings, los vectores de "rey" y "reina" estarán cerca.
- ### Relaciones semánticas y aritmética vectorial:
- ### Se pueden realizar operaciones matemáticas que reflejan relaciones semánticas, como:
- ### **rey−hombre+mujer≈reina**


## Clasificación de Textos por medio de Neuroevolución y Word Embeddings

<img src="figs/fig-diagrama-clasificador2.png" width="900">

# Entrenar al clasificador

### Clasificador: Red Neuronal Multicapa
- #### Define una red con una arquitectura que consta de:
    - #### 2 datos de entrada ($x_1$, $x_2$)
    - #### capa 1 (4 neuronas)
    - #### capa 2 (3 neuronas)
    - #### capa 3 (2 neuronas): 2 datos de salida ($y_1$, $y_2$)

<center>
<img src="figs/fig-red_neuronal.png" width="800" style="background-color:white;">
</center>



- #### Número de parámetros de la red:
    - #### Pesos en la capa 1: $w_{ij}^{(1)}$ = 8 (2 entradas x 4 neuronas) y   4 sesgos ($bias$) (1 de cada neurona)
    - #### Pesos en la capa 2: $w_{ij}^{(2)}$ = 12 (4 entradas [4 neuronas de la capa 1] x 3 neuronas) y  3 sesgos ($bias$) (1 de cada neurona)
    - #### Pesos en la capa 3: $w_{ij}^{(3)}$ = 6 (3 entradas [3 neuronas de la capa 2] x 2 neuronas) y  2 sesgos ($bias$) (1 de cada neurona)
    - #### Total de parámetros: 35 (pesos y $bias$)



### 1. Cargar los datos

In [1]:
import pandas as pd
import numpy as np

# Variables globales 
_NEURONAS = 128
_TASA_REDUCCION = 0.2
_N_CLASES = 3
_TIPO_WORD_EMBEDDINGS = "we_ft"   # word embeddings creados de documentos del español general: posibles valores {"we_ft", "we_mx", "we_es"}


dataset = pd.read_json("./data/dataset_polaridad_es_train_embeddings.json", lines=True)
#conteo de clases
print("Total de ejemplos de entrenamiento")
print(dataset.klass.value_counts())
# Extracción de los word embeddings y 
X = np.vstack(dataset[_TIPO_WORD_EMBEDDINGS].to_numpy())
# Extracción de las etiquetas o clases de entrenamiento
Y = dataset['klass'].to_numpy()

Total de ejemplos de entrenamiento
klass
neutral     1485
positive     968
negative     689
Name: count, dtype: int64


### 2. Codificar las categorías (clases)

In [2]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# Normalizar las etiquetas a una codificación ordinal para entrada del clasificador
Y_encoded= le.fit_transform(Y)
print("Clases:")
print(le.classes_)
print("Clases codificadas:")
print(le.transform(le.classes_))



Clases:
['negative' 'neutral' 'positive']
Clases codificadas:
[0 1 2]


### 3. Preparar los conjuntos de datos  (datasets) para entrenamiento y para probar el rendimiento del clasificador

In [3]:
# Dividir el conjunto de datos en conjunto de entrenamiento (80%) y conjunto de pruebas (20%)
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val =  train_test_split(X, Y_encoded, test_size=0.2, stratify=Y_encoded, random_state=42)

Y_train = Y_train[:, np.newaxis] # Agregar una dimensión adicional para representar 1 ejemplo de entrenamiento por fila
Y_val = Y_val[:, np.newaxis] # Agregar una dimensión adicional para representar 1 ejemplo de entrenamiento por fila

print(Y_train)

[[0]
 [1]
 [0]
 ...
 [1]
 [2]
 [0]]


### 4. Definición de la arquitectura de la red

In [4]:
from torch import nn
import numpy as np
# Definir la red neuronal en PyTorch heredando de la clase base de Redes Neuronales: Module
class RedNeuronal(nn.Module):
    def __init__(self, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion = 0.3):
        super().__init__()
        # Redondeado hacia arriba, reducir con la tasa de reducción "reduction_rate"
        self.tam_capa_oculta2 = int(tam_capa_oculta - np.ceil(tam_capa_oculta * tasa_reduccion))
        # print(f"tamaño capa oculta2: {self.tam_capa_oculta2 }")

        # Definición de capas, funciones de activación e inicialización de pesos
        # Capa Fully Connected (Capa Totalmente Conectada)
        self.fc1 = nn.Linear(tam_entrada, tam_capa_oculta)
        self.fc2 = nn.Linear(tam_capa_oculta, self.tam_capa_oculta2)
        self.fc3 = nn.Linear(self.tam_capa_oculta2, tam_salida)

        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.tam_capa_1_pesos = tam_entrada * tam_capa_oculta 
        self.tam_capa_1_bias  =  tam_capa_oculta # bias 
        self.tam_capa_2_pesos = tam_capa_oculta * self.tam_capa_oculta2 
        self.tam_capa_2_bias =  self.tam_capa_oculta2 # bias 
        self.tam_capa_3_pesos = self.tam_capa_oculta2 * tam_salida
        self.tam_capa_3_bias =  tam_salida # bias 
        self.tam_individuo = int(self.tam_capa_1_pesos + self.tam_capa_1_bias  +  self.tam_capa_2_pesos + self.tam_capa_2_bias + self.tam_capa_3_pesos + self.tam_capa_3_bias)
        # print(f"tamaño individuo (genoma): {self.tam_individuo  }")

    def forward(self, X):
        # Definición del orden de conexión de las capas y aplición de las funciones de activación
        out = self.fc1(X)
        out = self.relu(out)  # Aplicamos la función de activación
        out = self.fc2(out)
        out = self.relu(out)  # Aplicamos la función de activación
        out = self.fc3(out)
        out = self.softmax(out)  # Aplicamos la activación softmax para obtenr la probabilidad de cada neurona de salida
        return out

### 5. Definición del algoritmo evolutivo

In [5]:
import torch
import random
import numpy as np
from multiprocessing import  cpu_count
from sklearn.metrics import f1_score
from joblib import Parallel, delayed


#------------------------------------------------------------------------
#------------------------------------------------------------------------
# Funciones del algoritmo genético
#------------------------------------------------------------------------
#------------------------------------------------------------------------

def inicializar_poblacion(tam_poblacion, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion):
    # Cada individuo es del tamaño del total de los pesos y bias que conforman a la red neuronal
    # Un individuo está formado por los pesos y bias de todas sus capas en forma de un vector
    # indviduo = [w11(1), w12(1), w13(1), b1(1), b2(1), w11(2), w12(2, b1(2), b2(2), ....]
    # w11(1) representa el peso w11 de la capa 1
    # wij(n) representa un peso i,j de la capa n
    # bk(n) representa un sesgo (bias) k de la capa n
    
    red = RedNeuronal(tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion)    
    tam_individuo = red.tam_individuo
    poblacion = np.random.uniform(low=-0.5, high=0.5, size=(tam_poblacion, tam_individuo))
    return poblacion

#--------------------------------------------------------------------    
def inicializar_poblacion_xavier(tam_poblacion, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion):
    poblacion = []
    for k in range(tam_poblacion):        
        red = RedNeuronal(tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion)
        nn.init.xavier_normal_(red.fc1.weight)
        nn.init.xavier_normal_(red.fc2.weight)
        nn.init.xavier_normal_(red.fc3.weight)
        nn.init.xavier_normal_(red.fc1.bias)
        nn.init.xavier_normal_(red.fc2.bias)
        nn.init.xavier_normal_(red.fc3.bias)
        
        with torch.no_grad():
            individuo = np.concatenate((red.fc1.weight.flatten(), 
                        red.fc1.bias.flatten(),
                        red.fc2.weight.flatten(), 
                        red.fc2.bias.flatten(),
                        red.fc3.weight.flatten(), 
                        red.fc3.bias.flatten()))

            poblacion.append(individuo)
    poblacion = np.array(poblacion)        
    print(f"tamaño población: {poblacion.shape}")
    return poblacion

#--------------------------------------------------------------------
def ajustar_estructura_red_neuronal(individuo, red_neuronal, tam_entrada, tam_capa_oculta, tam_salida):
 # Cargar pesos del individuo a la estrcutura de la red neuronal
    with torch.no_grad():
        # La matriz de pesos en Pytorch tienen la forma transpuesta (salida, entradas) a la inversa como se define la capa Lineal
        # Se reestablece el vector de pesos y bias que representa a cada individuo a su estrcutura de la red neuronal
        # 1. Se extraen las secciones (slice) del individuo (vector) que corresponden a la capa1 (fc1) y se extraen los bias de la capa1 (fc1)
        # 1.2. Se asignan a las secciones correspondiente de la red (weight.data) y (bias.data). 
        # 1.3.  Deben tener la misma forma (shape) que la estructura de la red definida, de lo contrario indicará el error.
        # 2. Se repiten el proceso para las capas restantes: desplazándose la sección de los pesos y bias de la primera capa (fc1) y extraer
        #    los pesos y bias de la capa2 (fc2) y asignarlos a los parámetros correspondientes.
        red_neuronal.fc1.weight.data = torch.tensor(individuo[:red_neuronal.tam_capa_1_pesos].reshape(tam_capa_oculta, tam_entrada)).float()
        desplazamiento_capa_1 =   red_neuronal.tam_capa_1_pesos + red_neuronal.tam_capa_1_bias
        red_neuronal.fc1.bias.data = torch.tensor(individuo[red_neuronal.tam_capa_1_pesos:desplazamiento_capa_1]).float()
        
        red_neuronal.fc2.weight.data = torch.tensor(individuo[desplazamiento_capa_1:desplazamiento_capa_1 + red_neuronal.tam_capa_2_pesos].reshape(red_neuronal.tam_capa_oculta2, tam_capa_oculta)).float()
        desplazamiento_capa_2 = desplazamiento_capa_1 + red_neuronal.tam_capa_2_pesos + red_neuronal.tam_capa_2_bias
        red_neuronal.fc2.bias.data = torch.tensor(individuo[desplazamiento_capa_1 + red_neuronal.tam_capa_2_pesos:desplazamiento_capa_2]).float()
        
        red_neuronal.fc3.weight.data = torch.tensor(individuo[desplazamiento_capa_2:desplazamiento_capa_2 + red_neuronal.tam_capa_3_pesos].reshape(tam_salida, red_neuronal.tam_capa_oculta2)).float()
        desplazamiento_capa_3 = desplazamiento_capa_2 + red_neuronal.tam_capa_3_pesos + red_neuronal.tam_capa_3_bias
        red_neuronal.fc3.bias.data = torch.tensor(individuo[desplazamiento_capa_2 + red_neuronal.tam_capa_3_pesos:desplazamiento_capa_3]).float()
    return red_neuronal


#--------------------------------------------------------------------
#  Función para evaluar el individuo (que representa su genoma)
def funcion_fitness(individuo, X, Y, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion):
    red_neuronal = RedNeuronal(tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion)
    red_neuronal = ajustar_estructura_red_neuronal(individuo, red_neuronal, tam_entrada, tam_capa_oculta, tam_salida)

    # Calcular precisión

    X_tensor = torch.tensor(X).float()

    with torch.no_grad():

        y_pred = red_neuronal(X_tensor)
        # Obtiene una única clase, la más probable
        y_pred = torch.argmax(y_pred, dim=1)
        score = f1_score(Y, y_pred, average="macro")
            
    return score

#--------------------------------------------------------------------
def evaluar_fitness(poblacion,  X, Y, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion):
    fitness = []
    for individuo in poblacion:
        val_fitness = funcion_fitness(individuo, X, Y, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion)
        fitness.append(val_fitness)
    return  np.array(fitness)

    
#--------------------------------------------------------------------
def evaluar_fitness_paralelo(poblacion,  X, Y, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion, n_jobs=-1):
    """Evalúa la población en paralelo usando multiprocessing."""
    print(f"Evaluando población en paralelo ({n_jobs} núcleos)...")
    # Configurar el número de workers (n_jobs=-1 usa todos los núcleos)
    n_jobs = cpu_count() if n_jobs == -1 else n_jobs

    # Evaluar en paralelo

    resultados = Parallel(
    n_jobs = n_jobs-1,
    timeout = 300,  # 5min
    verbose = 10
    )(
        delayed(funcion_fitness)(individuo, X, Y, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion)
        for individuo in poblacion
    )

    fitness = np.array(resultados)
    
    return fitness
    
#------------------------------------------------------------------------
def seleccionar_padres(poblacion, aptitudes, k=5):
    """Selecciona dos padres mediante torneo."""
    torneo = random.sample(list(zip(poblacion, aptitudes)), k=k)
    torneo.sort(key=lambda x: x[1])  
    return torneo[0][0], torneo[1][0]

#------------------------------------------------------------------------
def elitismo(poblacion, fitness, tam_elite=2):
    """Selecciona los 'tam_elite' mejores best_individuos."""
    # Ordenar por fitness (mayor = mejor)
    ranked_indices = np.argsort(fitness)[::-1]
    elites = [poblacion[i] for i in ranked_indices[:tam_elite]]
    return elites


#------------------------------------------------------------------------
def cruzar(padre1, padre2):
    punto_cruza = np.random.randint(len(padre1))
    hijo1 = np.concatenate([padre1[:punto_cruza], padre2[punto_cruza:]])
    hijo2 = np.concatenate([padre2[:punto_cruza], padre1[punto_cruza:]])
    return hijo1, hijo2

def cruzar_uniforme(padre1, padre2):
    #Cruza uniforme
    probabilidad_intercambio=0.5
    # Generar máscara de intercambio
    mascara = np.random.random(len(padre1)) < probabilidad_intercambio
    # Crear hijos
    hijo1 = np.where(mascara, padre2, padre1)
    hijo2 = np.where(mascara, padre1, padre2)
    return hijo1, hijo2


#------------------------------------------------------------------------
def mutar(individuo, tasa_mutacion=0.1, sigma=1):
    # Crea la mascara de genes por mutar que cumplan con la condición
    mascara = np.random.rand(len(individuo)) < tasa_mutacion
    individuo += mascara * np.random.normal(0, sigma, size=len(individuo))  # Mutar al individuo en los genes seleccionados 
    return individuo

    
def algoritmo_evolutivo(X, Y, tam_poblacion=30, num_generaciones=50, run_paralelo=False):
    tam_entrada = X.shape[1]  # Total de Características: dimensiones del word embeddings
    tam_capa_oculta = _NEURONAS
    tam_salida = _N_CLASES
    tasa_reduccion = _TASA_REDUCCION
        
    # Población inicial
    poblacion = inicializar_poblacion(tam_poblacion, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion)
    # poblacion = inicializar_poblacion_xavier(tam_poblacion, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion)
    best_fitness_hist = []
    mean_fitness_hist = []

    for generacion in range(num_generaciones):
        # Evaluar fitness (usando el conjunto de entrenamiento)
        if run_paralelo:
            val_fitness = evaluar_fitness_paralelo(poblacion,  X, Y, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion, n_jobs=-1)
        else:            
            val_fitness = evaluar_fitness(poblacion, X, Y, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion)
        
        # Registrar estadísticas
        print("obteniendo mejor fitness" )
        best_fitness = np.max(val_fitness)
        print("obteniendo media fitness " )
        mean_fitness = np.mean(val_fitness)
        best_fitness_hist.append(best_fitness)
        mean_fitness_hist.append(mean_fitness)
        
        print(f"Generación {generacion + 1}: Mejor fitness = {best_fitness:.4f}, Fitness promedio = {mean_fitness:.4f}")
        nueva_poblacion = []

        for _ in range(tam_poblacion // 2):
            # Seleccionar padres
            padre1, padre2 = seleccionar_padres(poblacion, val_fitness)            
            hijo1, hijo2 = cruzar(padre1, padre2)
            hijo1 = mutar(hijo1)
            hijo2 = mutar(hijo2)
            nueva_poblacion.append(hijo1)            
            nueva_poblacion.append(hijo2)        
        
        
        #-----------------
        # Población: Los hijos sustituyen a los padres
        #-----------------
        # poblacion = np.array(nueva_poblacion)
    
        #-----------------
        # Población con elitismo de padres y parte de los hijos
        #-----------------
        nueva_poblacion = np.array(nueva_poblacion)
        K_best_padres = 10
        poblacion[:K_best_padres, ] = elitismo(poblacion, val_fitness, K_best_padres)
        poblacion[K_best_padres:, ] = nueva_poblacion[K_best_padres:, ]


    if run_paralelo:
        val_fitness = evaluar_fitness_paralelo(poblacion,  X, Y, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion, n_jobs=-1)
    else:            
        val_fitness = evaluar_fitness(poblacion, X, Y, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion)


    # Evaluar el mejor modelo en train
    best_individuo = poblacion[np.argmax(val_fitness)]
    test_f1 = funcion_fitness(best_individuo, X, Y, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion)
    print(f"\nF1-score final en test: {test_f1:.3f}")
    return best_individuo


def predecir_clase(individuo, X, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion):
    red_neuronal = RedNeuronal(tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion)
    red_neuronal = ajustar_estructura_red_neuronal(individuo, red_neuronal, tam_entrada, tam_capa_oculta, tam_salida)
 
    # Calcular precisión
    X_tensor = torch.from_numpy(X).float()

    with torch.no_grad():
        # Calcula las predicciones con la red neuronal con los pesos y bias definidos por el individuo
        y_pred = red_neuronal(X_tensor)
        y_pred = torch.argmax(y_pred, dim=1)
        return y_pred
    

### 7. Ejecución del algoritmo evolutivo

In [8]:

best_individuo = algoritmo_evolutivo(X_train, Y_train, tam_poblacion=50, num_generaciones=50, run_paralelo=False)

obteniendo mejor fitness
obteniendo media fitness 
Generación 1: Mejor fitness = 0.3023, Fitness promedio = 0.1906
obteniendo mejor fitness
obteniendo media fitness 
Generación 2: Mejor fitness = 0.3251, Fitness promedio = 0.2151
obteniendo mejor fitness
obteniendo media fitness 
Generación 3: Mejor fitness = 0.3730, Fitness promedio = 0.2133
obteniendo mejor fitness
obteniendo media fitness 
Generación 4: Mejor fitness = 0.3730, Fitness promedio = 0.2116
obteniendo mejor fitness
obteniendo media fitness 
Generación 5: Mejor fitness = 0.3730, Fitness promedio = 0.2166
obteniendo mejor fitness
obteniendo media fitness 
Generación 6: Mejor fitness = 0.3730, Fitness promedio = 0.1994
obteniendo mejor fitness
obteniendo media fitness 
Generación 7: Mejor fitness = 0.3730, Fitness promedio = 0.1943
obteniendo mejor fitness
obteniendo media fitness 
Generación 8: Mejor fitness = 0.3730, Fitness promedio = 0.1985
obteniendo mejor fitness
obteniendo media fitness 
Generación 9: Mejor fitness =

### 8. Prueba del mejor individuo en el conjunto de test

In [9]:
import numpy as np
import pandas as pd

tam_entrada = X_train.shape[1] # Tamaño del word embeddigns
tam_capa_oculta = _NEURONAS
tam_salida = _N_CLASES
tasa_reduccion = _TASA_REDUCCION

dataset_test = pd.read_json("./data/dataset_polaridad_es_test_embeddings.json", lines=True)
#conteo de clases
print("Total de ejemplos de entrenamiento")
print(dataset_test.klass.value_counts())
# Extracción de los textos en arreglos de numpy
X_test = np.vstack(dataset_test[_TIPO_WORD_EMBEDDINGS].to_numpy())
# Extracción de las etiquetas o clases de entrenamiento
Y_test = dataset_test['klass'].to_numpy()

Y_test = le.transform(Y_test)
Y_t = Y_test[:, np.newaxis] # Agregar una dimensión adicional para representar 1 ejemplo de entrenamiento por fila
y_pred_test = predecir_clase(best_individuo, X_test, tam_entrada, tam_capa_oculta, tam_salida, tasa_reduccion)
print(Y_t[:5])
print(y_pred_test[:5])
score = f1_score(Y_t, y_pred_test, average="macro")
print(f"\nF1-score final en test: {score:.3f}")



Total de ejemplos de entrenamiento
klass
neutral     371
positive    242
negative    173
Name: count, dtype: int64
[[2]
 [1]
 [0]
 [1]
 [1]]
tensor([2, 0, 2, 0, 2])

F1-score final en test: 0.388


# Ejercicio 1

### 1. Probar diferentes tipos de word embeddings.
- ### Campo del dataset "we_ft": Word Embeddings construidos con documentos del español general
- ### Campo del dataset "we_mx": Word Embeddings construidos con tuits del español de México
- ### Campo del dataset "we_es": Word Embeddings construidos con tuits del español de España
### 2. Probar diferentes tipos de operadores de variación:
- ### Cruza (1 punto, 2 puntos, n puntos)
### 3. Variar el tamaño de la población (considerar que en estos casos el costo computacional es mayor)
### 4. Variar el número de generaciones (considerar que en estos casos el costo computacional es mayor)
